In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from pystacknet.pystacknet import StackNetRegressor
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from fastFM import als, mcmc, sgd

from sklearn.metrics import mean_squared_error, mean_absolute_error

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP

import types

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
mytrial = list(pd.read_pickle('../trial/pystacknet.pkl').T.to_dict().values())
df_trial = pd.DataFrame(mytrial)
len(mytrial)

2

In [7]:
df_trial[['n_columns','train_mae','remark']]

,n_columns,train_mae,remark
0,30,1.721495,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
1,30,1.664118,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->CatBoostRegressor->


In [8]:
df_trial_catboost = pd.read_pickle('../trial/catboost.pkl')
df_trial_catboost.loc[452:452][['datetime','remark','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
452,2019-05-16 06:46:11.662876,tune 437,30,1.853478,0.000005,1.900535,0.000072,0.047056


In [9]:
df_trial_xgbm = pd.read_pickle('../trial/xgbm.pkl')
df_trial_xgbm.loc[1172:1172][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
1172,2019-05-16 09:18:54.750120,tune 1099,25,1.924779,0.000016,1.96556,0.000066,0.040782


In [10]:
df_trial_lgbm = pd.read_pickle('../trial/lgbm.pkl')
df_trial_lgbm.loc[2156:2156][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
2156,2019-05-16 22:30:28.865774,tune 2025,25,1.911171,0.00002,1.952897,0.000129,0.041726


In [11]:
df_trial_randomforest = pd.read_pickle('../trial/randomforest.pkl')
df_trial_randomforest.loc[297:297][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
297,2019-05-16 12:50:31.611238,tune 239,80,2.080347,0.000003,2.086208,0.000063,0.00586


In [12]:
df_trial_extratrees = pd.read_pickle('../trial/extratrees.pkl')
df_trial_extratrees.loc[459:459][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
459,2019-05-16 11:30:08.432873,tune 427,70,2.109147,0.000011,2.110347,0.000025,0.0012


In [13]:
df_trial_gradientboosting = pd.read_pickle('../trial/gradientboosting.pkl')
df_trial_gradientboosting.loc[306:306][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
306,2019-05-16 14:16:23.815368,tune 220,70,1.914761,0.000019,1.951837,0.000144,0.037076


In [14]:
df_trial_knn = pd.read_pickle('../trial/knn.pkl')
df_trial_knn.loc[17:17][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
17,2019-05-16 23:11:48.201772,None,7,2.026411,0.002031,2.081674,0.013139,0.055264


In [15]:
df_trial_svr = pd.read_pickle('../trial/svr.pkl')
df_trial_svr.loc[7:7][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
7,2019-05-16 23:57:06.120463,None,7,2.050366,0.003073,2.092532,0.01549,0.042166


In [16]:
df_trial_fm = pd.read_pickle('../trial/fm.pkl')
df_trial_fm.loc[0:0][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-15 05:53:03.716094,None,165,1.967943,0.000024,2.084029,0.000882,0.116085


In [17]:
df_trial_lasso = pd.read_pickle('../trial/lasso.pkl')
df_trial_lasso.loc[8:8][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
8,2019-05-16 11:54:52.999400,None,165,2.106014,0.002088,2.156098,0.008084,0.050085


In [18]:
df_trial_ridge = pd.read_pickle('../trial/ridge.pkl')
df_trial_ridge.loc[15:15][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
15,2019-05-16 11:27:58.322760,None,165,2.14516,0.002662,2.180493,0.012586,0.035332


In [19]:
def stacknet_params(models):
    model_names = []
    for lv_idx,models_current_lv  in enumerate(models):
        model_names.append([(model.__class__.__name__,model.get_params())   for model in models_current_lv])
    return model_names

def stacknet_remark(models):
    remark = ''
    for lv_idx,models_current_lv  in enumerate(models):
        remark += '|'.join([model.__class__.__name__ for model in models_current_lv])
        remark += '->'
        
    return remark

def process(df_train, df_test, columns, models, trial , remark=None, restacking=True, use_retraining=True, folds=3):
    
    X_train,y_train = df_train[columns].values, df_train['y'].values
    X_test = df_test[columns].values
    scaler = StandardScaler()
    scaler.fit(np.concatenate([X_train, X_test], axis=0))
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    stacknet = StackNetRegressor(models, metric="mae", folds=folds, restacking=restacking, use_retraining=use_retraining, random_state=12345, n_jobs=1, verbose=1)
    stacknet.fit(X_train,y_train)
    y_train_pred = stacknet.predict(X_train)
    y_test_pred = stacknet.predict(X_test)
    
    trial.append({'n_columns':len(columns), 'columns':columns, 'remark':stacknet_remark(models), 'models':stacknet_params(models), 'y_train_pred':y_train_pred, 'y_test_pred':y_test_pred, 'train_mae':mean_absolute_error(y_train, y_train_pred)})
    return stacknet


def process2(df_train, df_test, columns, models, splits):
    
    trial = []
    X_train,y_train = df_train[columns].values, df_train['y'].values
    X_test = df_test[columns].values
    scaler = StandardScaler()
    scaler.fit(np.concatenate([X_train, X_test], axis=0))
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    for fold_n, (train_index, valid_index) in enumerate(splits):
        for m in models:
            m.fit(X_train[train_index], y_train[train_index])
            y_train_pred = m.predict(X_train[train_index])
            y_valid_pred = m.predict(X_train[valid_index])
            metric_train = mean_absolute_error(y_train[train_index], y_train_pred)
            metric_valid = mean_absolute_error(y_train[valid_index], y_valid_pred)
            trial.append({'algorithm':m.__class__.__name__, 'fold_n':fold_n, 'n_columns':len(columns), 'metric_train':metric_train, 'metric_valid':metric_valid, 'y_train_pred':y_train_pred, 'y_valid_pred':y_valid_pred, 'columns':columns})
            
            
            
            
    return pd.DataFrame(trial)

In [22]:
columns_ = df_trial_catboost.loc[452]['param']['columns']+df_trial_xgbm.loc[1172]['param']['columns']+df_trial_lgbm.loc[2156]['param']['columns']+df_trial_randomforest.loc[297]['param']['columns']+df_trial_extratrees.loc[459]['param']['columns']+df_trial_gradientboosting.loc[306]['param']['columns']

unique_columns = list(set(columns_))
common_columns = []
main_columns = []
for col in unique_columns:
    if columns_.count(col)==6:
        common_columns.append(col)
    if columns_.count(col)>int(6*.8):
        main_columns.append(col)
len(unique_columns), len(common_columns),len(main_columns)
catboost_columns = df_trial_catboost.loc[452]['param']['columns'].copy()

In [23]:
trial_lst = [
    df_trial_catboost.loc[452], df_trial_xgbm.loc[1172], df_trial_lgbm.loc[2156],df_trial_gradientboosting.loc[306],
             df_trial_randomforest.loc[297],df_trial_extratrees.loc[459],
            df_trial_knn.loc[17], df_trial_svr.loc[0],
            df_trial_lasso.loc[8], df_trial_ridge.loc[15],
            ]
name_lst = [
    'catboost452', 'xgbm1172', 'lgbm2156',
    'gradientboosting306','randomforest297','extratrees459',
    'knn17','svr0','lasso8','ridge15',
    
]

In [29]:
models=[ 
######## First level ########
    [
        RandomForestRegressor (**df_trial_randomforest.loc[122]['param']['algorithm']['init']),
        ExtraTreesRegressor (**df_trial_extratrees.loc[250]['param']['algorithm']['init']),
        GradientBoostingRegressor(**df_trial_gradientboosting.loc[83]['param']['algorithm']['init']),
        cb.CatBoostRegressor(**df_trial_catboost.loc[342]['param']['algorithm']['init']),
        lgb.LGBMRegressor(**df_trial_lgbm.loc[1398]['param']['algorithm']['init']),
        xgb.XGBRegressor(**df_trial_xgbm.loc[865]['param']['algorithm']['init']),
#         KNeighborsRegressor(**df_trial_knn.loc[4]['param']['algorithm']['init']),
#         SVR(**df_trial_svr.loc[0]['param']['algorithm']['init']),
        Lasso(**df_trial_lasso.loc[4]['param']['algorithm']['init']),
        Ridge(**df_trial_ridge.loc[3]['param']['algorithm']['init']),
#         als.FMRegression(**df_trial_fm.loc[9]['param']['algorithm']['init']),
     ],
######## Second level ########
    [
        cb.CatBoostRegressor(**df_trial_catboost.loc[342]['param']['algorithm']['init']),
#         xgb.XGBRegressor(**df_trial_xgbm.loc[865]['param']['algorithm']['init']),
#         RandomForestRegressor (**df_trial_randomforest.loc[122]['param']['algorithm']['init']),
#         SVR(**df_trial_svr.loc[0]['param']['algorithm']['init']),
#         Ridge(**df_trial_ridge.loc[3]['param']['algorithm']['init']),
    ],
######## third level ########
#     [
#         Ridge(**df_trial_ridge.loc[3]['param']['algorithm']['init'])
#     ]
]

In [30]:
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

kfold_type = 'StratifiedKFold'
n_splits = 3

if kfold_type == 'StratifiedKFold':
    folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    splits = list(folds.split(df_train, df_train['label']))
elif kfold_type == 'GroupKFold':
    folds = GroupKFold(n_splits=3)
    splits = list(folds.split(df_train, groups=df_train['group']))
else:
    folds = KFold(n_splits=3, shuffle=True, random_state=42)
    splits = list(folds.split(df_train))

In [55]:
# mytrial = []

In [84]:
# df_trial2 = process2(df_train, df_test, columns, models[0],splits=splits)

In [50]:
# df_trial2['n_columns'] = df_trial2['columns'].apply(lambda x: len(x))
# df_trial2['metric_diff'] =  df_trial2['metric_valid'] - df_trial2['metric_train']

In [85]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(25,5))
# N_ = splits[0][1].shape[0]
# sns.lineplot(x=np.arange(N_) ,y=df_train['y'].values[splits[0][1]])
# sns.lineplot(x=np.arange(N_) ,y=df_trial2[(df_trial2['algorithm']=='KNeighborsRegressor')&(df_trial2['fold_n']==0)]['y_valid_pred'].values[0])

In [86]:
# df_trial2[['algorithm', 'fold_n', 'n_columns', 'metric_train', 'metric_valid', 'metric_diff']].sort_values(by=['algorithm','fold_n'])

In [87]:
# df_trial2[(df_trial2['fold_n']==0)][['algorithm', 'fold_n', 'n_columns', 'metric_train', 'metric_valid', 'metric_diff']].sort_values(by=['algorithm','fold_n'])

In [71]:
# sns.pairplot(pd.DataFrame(np.array(df_trial2[(df_trial2['fold_n']==0)]['y_valid_pred'].tolist()).T, columns=df_trial2[(df_trial2['fold_n']==0)]['algorithm'].tolist()))

In [ ]:
columns = catboost_columns #df_trial_catboost.loc[342]['param']['columns']
stacknet = process(df_train, df_test, columns, models, mytrial, folds=splits)

====================== Start of Level 0 ======================
Input Dimensionality 30 at Level 0 
8 models included in Level 0 
0:	learn: 6.4507175	total: 14.3ms	remaining: 8.39s
1:	learn: 6.1768644	total: 31.6ms	remaining: 9.27s
2:	learn: 5.9185205	total: 48ms	remaining: 9.37s
3:	learn: 5.6763596	total: 63.2ms	remaining: 9.25s
4:	learn: 5.4491238	total: 78.3ms	remaining: 9.14s
5:	learn: 5.2366465	total: 94.6ms	remaining: 9.2s
6:	learn: 5.0380073	total: 112ms	remaining: 9.27s
7:	learn: 4.8521888	total: 129ms	remaining: 9.35s
8:	learn: 4.6786439	total: 144ms	remaining: 9.31s
9:	learn: 4.5162535	total: 159ms	remaining: 9.23s
10:	learn: 4.3654408	total: 174ms	remaining: 9.14s
11:	learn: 4.2248975	total: 192ms	remaining: 9.21s
12:	learn: 4.0942669	total: 208ms	remaining: 9.23s
13:	learn: 3.9727030	total: 225ms	remaining: 9.23s
14:	learn: 3.8604188	total: 243ms	remaining: 9.29s
15:	learn: 3.7564547	total: 260ms	remaining: 9.32s
16:	learn: 3.6598624	total: 278ms	remaining: 9.35s
17:	learn: 

158:	learn: 2.4548403	total: 2.51s	remaining: 6.8s
159:	learn: 2.4544371	total: 2.53s	remaining: 6.79s
160:	learn: 2.4537764	total: 2.54s	remaining: 6.77s
161:	learn: 2.4534715	total: 2.56s	remaining: 6.75s
162:	learn: 2.4526687	total: 2.58s	remaining: 6.74s
163:	learn: 2.4516724	total: 2.59s	remaining: 6.72s
164:	learn: 2.4511747	total: 2.61s	remaining: 6.71s
165:	learn: 2.4506224	total: 2.63s	remaining: 6.7s
166:	learn: 2.4504502	total: 2.64s	remaining: 6.68s
167:	learn: 2.4498111	total: 2.66s	remaining: 6.66s
168:	learn: 2.4495909	total: 2.67s	remaining: 6.64s
169:	learn: 2.4485617	total: 2.69s	remaining: 6.64s
170:	learn: 2.4478797	total: 2.71s	remaining: 6.63s
171:	learn: 2.4471272	total: 2.73s	remaining: 6.61s
172:	learn: 2.4465218	total: 2.74s	remaining: 6.59s
173:	learn: 2.4463093	total: 2.76s	remaining: 6.58s
174:	learn: 2.4457684	total: 2.77s	remaining: 6.56s
175:	learn: 2.4448178	total: 2.79s	remaining: 6.55s
176:	learn: 2.4438906	total: 2.8s	remaining: 6.53s
177:	learn: 2.4

317:	learn: 2.3766753	total: 5.1s	remaining: 4.34s
318:	learn: 2.3763734	total: 5.11s	remaining: 4.33s
319:	learn: 2.3760630	total: 5.13s	remaining: 4.31s
320:	learn: 2.3759490	total: 5.14s	remaining: 4.29s
321:	learn: 2.3756687	total: 5.16s	remaining: 4.28s
322:	learn: 2.3754930	total: 5.18s	remaining: 4.26s
323:	learn: 2.3752184	total: 5.19s	remaining: 4.25s
324:	learn: 2.3744317	total: 5.21s	remaining: 4.23s
325:	learn: 2.3741523	total: 5.22s	remaining: 4.21s
326:	learn: 2.3739176	total: 5.24s	remaining: 4.2s
327:	learn: 2.3734729	total: 5.26s	remaining: 4.18s
328:	learn: 2.3731007	total: 5.28s	remaining: 4.17s
329:	learn: 2.3727523	total: 5.29s	remaining: 4.15s
330:	learn: 2.3722765	total: 5.31s	remaining: 4.14s
331:	learn: 2.3720759	total: 5.33s	remaining: 4.12s
332:	learn: 2.3718166	total: 5.34s	remaining: 4.11s
333:	learn: 2.3713708	total: 5.36s	remaining: 4.09s
334:	learn: 2.3710832	total: 5.38s	remaining: 4.08s
335:	learn: 2.3706644	total: 5.39s	remaining: 4.06s
336:	learn: 2.

476:	learn: 2.3274115	total: 7.68s	remaining: 1.8s
477:	learn: 2.3273422	total: 7.7s	remaining: 1.79s
478:	learn: 2.3270506	total: 7.72s	remaining: 1.77s
479:	learn: 2.3269034	total: 7.73s	remaining: 1.75s
480:	learn: 2.3268079	total: 7.75s	remaining: 1.74s
481:	learn: 2.3265294	total: 7.76s	remaining: 1.72s
482:	learn: 2.3262193	total: 7.78s	remaining: 1.71s
483:	learn: 2.3258286	total: 7.8s	remaining: 1.69s
484:	learn: 2.3253116	total: 7.81s	remaining: 1.68s
485:	learn: 2.3248801	total: 7.83s	remaining: 1.66s
486:	learn: 2.3247804	total: 7.84s	remaining: 1.64s
487:	learn: 2.3247392	total: 7.86s	remaining: 1.63s
488:	learn: 2.3244716	total: 7.88s	remaining: 1.61s
489:	learn: 2.3242949	total: 7.89s	remaining: 1.59s
490:	learn: 2.3238891	total: 7.91s	remaining: 1.58s
491:	learn: 2.3237921	total: 7.92s	remaining: 1.56s
492:	learn: 2.3236434	total: 7.94s	remaining: 1.55s
493:	learn: 2.3233995	total: 7.96s	remaining: 1.53s
494:	learn: 2.3231062	total: 7.97s	remaining: 1.51s
495:	learn: 2.3

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.



...

154:	learn: 2.4636231	total: 2.64s	remaining: 7.39s
155:	learn: 2.4630842	total: 2.66s	remaining: 7.38s
156:	learn: 2.4623533	total: 2.67s	remaining: 7.35s
157:	learn: 2.4613179	total: 2.69s	remaining: 7.34s
158:	learn: 2.4600858	total: 2.71s	remaining: 7.32s
159:	learn: 2.4593201	total: 2.72s	remaining: 7.29s
160:	learn: 2.4589440	total: 2.73s	remaining: 7.27s
161:	learn: 2.4580209	total: 2.75s	remaining: 7.25s
162:	learn: 2.4572537	total: 2.77s	remaining: 7.23s
163:	learn: 2.4562050	total: 2.78s	remaining: 7.22s
164:	learn: 2.4557661	total: 2.8s	remaining: 7.2s
165:	learn: 2.4551717	total: 2.82s	remaining: 7.18s
166:	learn: 2.4547151	total: 2.83s	remaining: 7.16s
167:	learn: 2.4544470	total: 2.85s	remaining: 7.14s
168:	learn: 2.4537621	total: 2.86s	remaining: 7.12s
169:	learn: 2.4531268	total: 2.88s	remaining: 7.1s
170:	learn: 2.4519813	total: 2.89s	remaining: 7.08s
171:	learn: 2.4513571	total: 2.91s	remaining: 7.06s
172:	learn: 2.4511469	total: 2.93s	remaining: 7.04s
173:	learn: 2.4

313:	learn: 2.3774174	total: 5.17s	remaining: 4.53s
314:	learn: 2.3767276	total: 5.19s	remaining: 4.51s
315:	learn: 2.3764049	total: 5.2s	remaining: 4.49s
316:	learn: 2.3760448	total: 5.22s	remaining: 4.48s
317:	learn: 2.3757271	total: 5.23s	remaining: 4.46s
318:	learn: 2.3756178	total: 5.25s	remaining: 4.44s
319:	learn: 2.3753097	total: 5.26s	remaining: 4.42s
320:	learn: 2.3750599	total: 5.28s	remaining: 4.41s
321:	learn: 2.3745827	total: 5.29s	remaining: 4.39s
322:	learn: 2.3742820	total: 5.31s	remaining: 4.37s
323:	learn: 2.3740016	total: 5.33s	remaining: 4.36s
324:	learn: 2.3737547	total: 5.34s	remaining: 4.34s
325:	learn: 2.3736164	total: 5.36s	remaining: 4.32s
326:	learn: 2.3732882	total: 5.37s	remaining: 4.3s
327:	learn: 2.3728424	total: 5.39s	remaining: 4.29s
328:	learn: 2.3725231	total: 5.4s	remaining: 4.27s
329:	learn: 2.3722625	total: 5.42s	remaining: 4.25s
330:	learn: 2.3719500	total: 5.44s	remaining: 4.24s
331:	learn: 2.3717267	total: 5.45s	remaining: 4.22s
332:	learn: 2.3

472:	learn: 2.3332476	total: 7.7s	remaining: 1.89s
473:	learn: 2.3330071	total: 7.72s	remaining: 1.87s
474:	learn: 2.3326595	total: 7.74s	remaining: 1.86s
475:	learn: 2.3325379	total: 7.75s	remaining: 1.84s
476:	learn: 2.3324307	total: 7.77s	remaining: 1.82s
477:	learn: 2.3322722	total: 7.78s	remaining: 1.81s
478:	learn: 2.3320686	total: 7.8s	remaining: 1.79s
479:	learn: 2.3319591	total: 7.82s	remaining: 1.77s
480:	learn: 2.3314536	total: 7.83s	remaining: 1.76s
481:	learn: 2.3312825	total: 7.85s	remaining: 1.74s
482:	learn: 2.3312206	total: 7.86s	remaining: 1.73s
483:	learn: 2.3308052	total: 7.88s	remaining: 1.71s
484:	learn: 2.3305897	total: 7.9s	remaining: 1.69s
485:	learn: 2.3303587	total: 7.92s	remaining: 1.68s
486:	learn: 2.3303015	total: 7.93s	remaining: 1.66s
487:	learn: 2.3300838	total: 7.95s	remaining: 1.64s
488:	learn: 2.3295642	total: 7.96s	remaining: 1.63s
489:	learn: 2.3294150	total: 7.98s	remaining: 1.61s
490:	learn: 2.3291590	total: 8s	remaining: 1.6s
491:	learn: 2.32911

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.



Fold 2/3 , model 0 , mae===2.094959 
Fold 2/3 , model 1 , mae===2.102836 
Fold 2/3 , model 2 , mae===1.914998 
Fold 2/3 , model 3 , mae===1.874953 
Fold 2/3 , model 4 , mae===1.897891 
Fold 2/3 , model 5 , mae===1.914108 
Fold 2/3 , model 6 , mae===2.116290 
Fold 2/3 , model 7 , mae===2.106869 
=========== end of fold 2 in level 0 ===========


In [ ]:
df_trial = pd.DataFrame(mytrial)
df_trial[['n_columns','train_mae','remark']]

In [99]:
df_trial.to_pickle('../trial/pystacknet.pkl')

In [42]:
idx = 2
y_test_pred = df_trial.loc[idx]['y_test_pred'].reshape(-1)
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = y_test_pred
df_submit['seg_id'] = df_test['index']
df_submit.to_csv('submission_pystacknet_{}.csv'.format(idx), index=False)

In [98]:
# df_trial = pd.DataFrame(mytrial)
# df_trial[['n_columns','train_mae','remark']]

,n_columns,train_mae,remark
0,30,1.721495,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
1,30,1.664118,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
2,30,1.657385,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
3,30,1.897036,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
4,30,1.897036,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
5,30,1.664747,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->XGBRegressor|RandomForestRegressor|SVR|Ridge->Ridge->
6,30,1.723844,CatBoostRegressor|LGBMRegressor|XGBRegressor->CatBoostRegressor->
7,30,1.720960,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|Lasso|Ridge->CatBoostRegressor->
8,30,1.735455,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|Lasso|Ridge->XGBRegressor->
9,165,1.463634,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|Lasso|Ridge->CatBoostRegressor->
